In [1]:
from arcgis.gis import GIS, Item
import os
from multiprocessing import Process, Queue

import sys
sys.path.append('../src')
sys.path.append('../')

import reach_tools
import hydrology
import config

gis = GIS(username=config.arcgis_username, password=config.arcgis_password)

url_reach_line = config.url_reach_line
url_reach_centroid = config.url_reach_centroid
url_reach_points = config.url_reach_points

lyr_line = reach_tools.ReachFeatureLayer(url_reach_line, gis)
lyr_centroid = reach_tools.ReachFeatureLayer(url_reach_centroid, gis)
lyr_points = reach_tools.ReachPointFeatureLayer(url_reach_points, gis)

%load_ext autoreload
%autoreload 2

In [2]:
reach_id_fs = lyr_centroid.query(out_fields='reach_id', return_geometry=False)
reach_id_lst = [int(feat.attributes['reach_id']) for feat in reach_id_fs.features]
print(reach_id_lst[:20])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


In [4]:
gis = GIS(username=config.arcgis_username, password=config.arcgis_password)

for reach_id in reach_id_lst:
    
    reach = reach_tools.Reach.get_from_aw(reach_id)
    reach.snap_putin_and_takeout_and_trace()
    reach.publish_updates(lyr_line, lyr_centroid, lyr_points)
        
    print(f'success: {reach.reach_id} {reach.river_name} {reach.reach_name} http://knu2xs.maps.arcgis.com/apps/View/index.html?appid=62b2a133c55a4a8c84f9aaa4ef461de0&reachid={reach.reach_id}')

success: 1 Alsek Haines Junction to Dry Bay http://knu2xs.maps.arcgis.com/apps/View/index.html?appid=62b2a133c55a4a8c84f9aaa4ef461de0&reachid=1
success: 2 Aniakchak Surprise Lake to Aniakchak Bay http://knu2xs.maps.arcgis.com/apps/View/index.html?appid=62b2a133c55a4a8c84f9aaa4ef461de0&reachid=2
success: 3 Browns Creek Browns Creek Canyon http://knu2xs.maps.arcgis.com/apps/View/index.html?appid=62b2a133c55a4a8c84f9aaa4ef461de0&reachid=3
success: 4 Canyon Creek Seward Highwy at outlet of Lower Summit Lake to Seward Hwy at Hope Junction http://knu2xs.maps.arcgis.com/apps/View/index.html?appid=62b2a133c55a4a8c84f9aaa4ef461de0&reachid=4
success: 5 Charley Upper river gravel bar to Yukon River to Circle http://knu2xs.maps.arcgis.com/apps/View/index.html?appid=62b2a133c55a4a8c84f9aaa4ef461de0&reachid=5
success: 6 Chetaslina Chichokna River to Copper River http://knu2xs.maps.arcgis.com/apps/View/index.html?appid=62b2a133c55a4a8c84f9aaa4ef461de0&reachid=6
success: 7 Chickaloon 30 Mile Airstrip 

TypeError: '<' not supported between instances of 'NoneType' and 'float'

In [51]:
from arcgis.geometry import Geometry
fs = lyr_line.query_by_reach_id(1)
geom = fs.features[0].geometry
geom = Geometry(geom)

In [56]:
geom.coordinates().size

7080

In [50]:
webmap = gis.map()
webmap.draw(geom)
webmap.extent = geom.extent
webmap

MapView(layout=Layout(height='400px', width='100%'))

In [45]:
from arcgis.gis import GIS
import requests
import json
from urllib.parse import quote

# create the densify url by accessing the definted Web GIS geometry service
url = GIS().properties.helperServices.geometry.url + '/densify'

# using this to get the key for the geometry containing the coordinates since the Geometry.JSON function does not work
geom_key = list(geom.keys())[0]

# create the payload for the post request
params = {
    'f': 'json',
    'geometries': {
        'geometryType': 'esriGeometryPolyline',
        'geometries': [{geom_key:geom[geom_key]}]
    },
    'sr': {'wkid': 4326},
    'lengthUnit': 9001,  # meters
    'maxSegmentLength': 180  # for 90m SRTM
}

# convert params to all string
payload = {k: json.dumps(v) if isinstance(v ,(dict, list)) else v for k, v in params.items()}

# use urlencode to make everything web friendly for sending
# encoded_payload = quote(payload, safe='')

# push the request
resp = requests.post(url, payload)

# extract the densified geometries
resp.content

b'{"geometryType":"esriGeometryPolyline","geometries":[{"paths":[[[-137.50522326999999,60.746811028000103],[-137.50626502099999,60.746807825000097],[-137.507306779,60.746804662000002],[-137.50834854600001,60.746801539000003],[-137.50939032100001,60.746798457000097],[-137.510432103,60.746795415999998],[-137.51147389400001,60.746792415000101],[-137.51251569300001,60.746789454000101],[-137.51355749999999,60.746786534000101],[-137.514599315,60.746783655000101],[-137.51564113800001,60.746780816000097],[-137.51668296899999,60.746778017000103],[-137.517724808,60.746775259000103],[-137.51876665500001,60.746772542000002],[-137.51980850999999,60.746769864999997],[-137.52085037399999,60.746767228000003],[-137.521892245,60.746764632000101],[-137.52293412399999,60.746762076000003],[-137.523976011,60.746759560999998],[-137.52501790700001,60.746757086000002],[-137.52605980999999,60.746754652000099],[-137.527101722,60.7467522580001],[-137.52814364100001,60.746749905000101],[-137.52918556899999,60.7467

In [43]:
payload

{'f': '"json"',
 'geometries': '{"geometryType": "esriGeometryPolyline", "geometries": [{"paths": [[[-137.50522327, 60.7468110280001], [-137.506265021, 60.7468078250001], [-137.507306779, 60.746804662], [-137.508348546, 60.746801539], [-137.509390321, 60.7467984570001], [-137.510432103, 60.746795416], [-137.511473894, 60.7467924150001], [-137.512515693, 60.7467894540001], [-137.5135575, 60.7467865340001], [-137.514599315, 60.7467836550001], [-137.515641138, 60.7467808160001], [-137.516682969, 60.7467780170001], [-137.517724808, 60.7467752590001], [-137.518766655, 60.746772542], [-137.51980851, 60.746769865], [-137.520850374, 60.746767228], [-137.521892245, 60.7467646320001], [-137.522934124, 60.746762076], [-137.523976011, 60.746759561], [-137.525017907, 60.746757086], [-137.52605981, 60.7467546520001], [-137.527101722, 60.7467522580001], [-137.528143641, 60.7467499050001], [-137.529185569, 60.7467475920001], [-137.530227504, 60.7467453200001], [-137.531269448, 60.7467430880001], [-137